# Overview
In this notebook, I used negex to detect negated entities. Negex was developed in early 2000', but it continues to be the leading package for negation detection. It was espcially developed to handle clinical text. We compare negex with stanza by using the spacy_stanza wrapper. Together, they will identify medical entities, and tell us whether each entity is negated.

# Use environment: transformer

In [1]:
from datetime import datetime
date = datetime.today().strftime('%y%m%d')
print ('Last modified by Xiaoqing: ' + date)

Last modified by Xiaoqing: 211203


# Installation

In [2]:
# import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m pip install negspacy
# !{sys.executable} -m pip install spacy_stanza #This package wraps the Stanza (formerly StanfordNLP) library, so you can use Stanford's models in a spaCy pipeline.

In [3]:
import spacy
import stanza
import spacy_stanza # this package wraps the Stanza (formerly StanfordNLP) library, so you can use Stanford's models in a spaCy pipeline.
from negspacy.negation import Negex
from negspacy.termsets import termset # to modify negation terms
import pandas as pd



# Set up pipeline, add customized terms to the default list of terms

In [22]:
# download and initialize a mimic pipeline with an i2b2 NER model
# stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = spacy_stanza.load_pipeline('en', package='mimic', processors={'ner': 'i2b2'})

ts = termset("en_clinical")
# customize the term list by adding more negation terms
ts.add_patterns({
            'preceding_negations': ['other than','except for','except','with the exception of',
                                    'excluding','lack of','contraindication','contraindicated','interfere with'],
            'following_negations':['negative']
        })
# print(ts.get_patterns()) # view negation terms in use

2021-12-03 12:43:23 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

2021-12-03 12:43:23 INFO: Use device: cpu
2021-12-03 12:43:23 INFO: Loading: tokenize
2021-12-03 12:43:23 INFO: Loading: pos
2021-12-03 12:43:23 INFO: Loading: lemma
2021-12-03 12:43:23 INFO: Loading: depparse
2021-12-03 12:43:23 INFO: Loading: ner
2021-12-03 12:43:24 INFO: Done loading processors!


In [23]:
nlp.add_pipe("negex", config={"ent_types":["PROBLEM","TEST",'TREATMENT']}) # let negex know what entities we are extracting


# Good examples

In [6]:
doc = nlp('patient has a fever, but no headache')

for e in doc.ents:
	print(e.text, e._.negex)

a fever False
headache True


In [26]:
doc = nlp('No history of diabetes')

for e in doc.ents:
	print(e.text, e._.negex)

diabetes True


In [20]:
doc = nlp('Patients who have a contraindication to DUPIXENT')

for e in doc.ents:
	print(e.text, e._.negex)

DUPIXENT True


In [24]:
doc = nlp('In the case of women of childbearing age, urine pregnancy test must be negative')

for e in doc.ents:
	print(e.text, e._.negex)

urine pregnancy test True


In [25]:
doc = nlp('History of or ongoing inflammatory, autoimmune, or painful musculoskeletal diseases (except for Sjogrens syndrome or fibromyalgia) which could interfere with the RA assessments as determined by the Investigator')

for e in doc.ents:
	print(e.text, e._.negex)

ongoing inflammatory, autoimmune False
painful musculoskeletal diseases False
Sjogrens syndrome True
fibromyalgia True
the RA assessments True


In [16]:
doc = nlp('History of autoimmune disorders other than T1DM as judged clinically relevant by the Investigator (obtained by patient history), except a stable thyroid disorder treated with a stable dose of thyroxin.')

for e in doc.ents:
	print(e.text, e._.negex)

autoimmune disorders False
T1DM True
a stable thyroid disorder True
thyroxin True


# Note to future: if a term is negated under exclusion criteria, we delete them

# References
Negation
* https://github.com/jenojp/negspacy
* https://medium.com/@MansiKukreja/clinical-text-negation-handling-using-negspacy-and-scispacy-233ce69ab2ac
* https://towardsdatascience.com/clinical-notes-the-negative-story-e1140dd275c7
* https://www.youtube.com/watch?v=IiD3YZkkCmE&t=2210s
(see 36:41 of the video)

Negex: how to add and delete custom negation terms
* https://pypi.org/project/negspacy/
